# Asset Management & Sustainable Finance
- Sarahnour Ghaith
- Thomas Roiseux

In [ ]:
import cvxpy as cp
import numpy as np

import matplotlib.pyplot as plt
from scipy.stats import norm

from typing import Dict, Callable

import warnings

warnings.filterwarnings("ignore")

## Exercise 1 - Portfolio optimization and risk budgeting

In [ ]:
corr_mat = np.array(
    [
        [1, 0.5, 0.3, 0.6, 0.4],
        [0.5, 1, 0.3, 0.6, 0.3],
        [0.3, 0.3, 1, 0.6, 0.7],
        [0.6, 0.6, 0.6, 1, 0.3],
        [0.4, 0.3, 0.7, 0.3, 1],
    ],
    dtype=np.float64,
)

r = 0.02  # risk free rate

mu = np.array([0.05, 0.05, 0.06, 0.04, 0.07], dtype=np.float64).T
sigma = np.array([0.2, 0.22, 0.25, 0.18, 0.45], dtype=np.float64).T

### Question 1.a

In [ ]:
cov_mat = np.zeros_like(corr_mat)  # Covariance matrix

for i in range(len(cov_mat)):
    for j in range(i, len(cov_mat)):
        cov_mat[i, j] = corr_mat[i, j] * sigma[i] * sigma[j]
        cov_mat[j, i] = cov_mat[i, j]


print(cov_mat)

### Question 1.b

In [ ]:
sr = (mu - r) / sigma  # Sharpe ratio

for k in range(len(mu)):
    print("Sharpe ratio for asset", k + 1, ":", sr[k])

### Question 2.b

In [ ]:
def solve_qp_problem(gamma: float) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        gamma (float): gamma parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    x = cp.Variable(len(mu), "x")  # Portfolio weights

    objective = cp.Minimize(
        0.5 * cp.quad_form(x, cov_mat) - gamma * cp.matmul(mu - r, x)
    )
    constraints = [cp.sum(x) == 1, -10 <= x, x <= 10]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return x.value

In [ ]:
optimal_x: Dict[float, np.ndarray] = {}
gammas = [0, 0.1, 0.2, 0.5, 1]

for gamma in gammas:
    sol = solve_qp_problem(gamma)
    print("Optimal portfolio for gamma =", gamma, ":", sol)
    optimal_x[gamma] = sol

In [ ]:
for gamma, sol in optimal_x.items():
    print("Gamma:", gamma)
    expected_return = np.dot(sol.T, mu)
    volatility = sol.T.dot(cov_mat).dot(sol)
    sharpe_ratio = (expected_return - r) / volatility
    print("  Expected return:", expected_return)
    print("  Volatility:", volatility)
    print("  Sharpe ratio:", sharpe_ratio)

### Question 2.c

In [ ]:
precise_gammas = np.linspace(-10, 10, 500)

solutions = [solve_qp_problem(gamma) for gamma in precise_gammas]
returns = np.array([np.dot(sol.T, mu) * 100 for sol in solutions])
volatilities = np.array([sol.T.dot(cov_mat).dot(sol) * 100 for sol in solutions])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(
    volatilities,
    returns,
    label="Efficient frontier",
)
plt.legend()
plt.grid(visible=True)
plt.xlabel("Volatility (in %)")
plt.ylabel("Expected return (in %)")
plt.title("Efficient frontier")
plt.show()

### Question 2.d

In [ ]:
def bisection_algorithm(
    f: Callable[[float], float],
    a: float,
    b: float,
    target: float,
    /,
    tol: float = 10**-5,
    *,
    max_iter: int = 100,
) -> float:
    """Bisection algorithm

    Args:
        f (function): function to find the root of
        a (float): left bound
        b (float): right bound
        target (float): target value
        tol (float, optional): tolerance. Defaults to 10**-5.
        max_iter (int, optional): maximum number of iterations. Defaults to 100.

    Returns:
        float: root of the function
    """
    if b - a < 0:
        a, b = b, a

    gamma_bar = (a + b) / 2

    if b - a < tol or max_iter == 0:
        return gamma_bar

    f_bar = f(gamma_bar)

    if f_bar < target:
        a = gamma_bar
    else:
        b = gamma_bar

    return bisection_algorithm(f, a, b, target, tol=tol, max_iter=max_iter - 1)

In [ ]:
targets = (16, 20)


def volatilities_function(gamma: float) -> float:
    sol = solve_qp_problem(gamma)
    return sol.T.dot(cov_mat).dot(sol) * 100


for target in targets:
    gamma = bisection_algorithm(
        volatilities_function,
        0,
        100,
        target,
    )
    print("Gamma for target volatility of", target, ":", gamma)
    print("  Expected return:", np.dot(solve_qp_problem(gamma).T, mu) * 100)
    print("  Volatility:", volatilities_function(gamma))
    print(
        "  Sharpe ratio:",
        (np.dot(solve_qp_problem(gamma).T, mu) - r) / volatilities_function(gamma),
    )

### Question 2.e

In [ ]:
sharpe_ratios = (returns - r * 100) / volatilities  # Converting r into %

i = np.argmax(sharpe_ratios)
tp = solve_qp_problem(precise_gammas[i])
print("Optimal portfolio for maximum Sharpe ratio:", tp)
print("Maximum Sharpe ratio:", sharpe_ratios[i])
print("Expected return (in %):", returns[i])
print("Volatility (in %):", volatilities[i])

### Question 2.f
The analytical solutoin can be found by maximizing the Sharpe Ratio. This is exactly what we did in the previous question so we will have the same answer.

### Question 3.a

In [ ]:
def solve_long_only_qp_problem(gamma: float) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        gamma (float): gamma parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    x = cp.Variable(len(mu), "x")  # Portfolio weights

    objective = cp.Minimize(
        0.5 * cp.quad_form(x, cov_mat) - gamma * cp.matmul(mu - r, x)
    )
    constraints = [cp.sum(x) == 1, 0 <= x, x <= 1]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return x.value

In [ ]:
for gamma in gammas:
    sol = solve_long_only_qp_problem(gamma)
    print("Optimal portfolio for gamma =", gamma, ":", sol)
    print("Gamma:", gamma)
    expected_return = np.dot(sol.T, mu)
    volatility = sol.T.dot(cov_mat).dot(sol)
    sharpe_ratio = (expected_return - r) / volatility
    print("  Expected return (in %):", expected_return * 100)
    print("  Volatility (in %):", volatility * 100)
    print("  Sharpe ratio:", sharpe_ratio)

### Question 3.b

In [ ]:
long_solutions = [solve_long_only_qp_problem(gamma) for gamma in precise_gammas]
long_returns = np.array([np.dot(sol.T, mu) * 100 for sol in long_solutions])
long_volatilities = np.array(
    [sol.T.dot(cov_mat).dot(sol) * 100 for sol in long_solutions]
)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(
    volatilities,
    returns,
    label="Efficient frontier long long/short",
)
plt.plot(
    long_volatilities,
    long_returns,
    label="Efficient frontier long only",
)
plt.legend()
plt.grid(visible=True)
plt.xlabel("Volatility (in %)")
plt.ylabel("Expected return (in %)")
plt.title("Efficient frontier")
plt.show()

### Question 3.c

In [ ]:
for target in targets:
    gamma = bisection_algorithm(
        volatilities_function,
        0,
        100,
        target,
    )
    print("Gamma for target volatility of", target, ":", gamma)
    print("  Expected return:", np.dot(solve_long_only_qp_problem(gamma).T, mu) * 100)
    print("  Volatility:", volatilities_function(gamma))
    print(
        "  Sharpe ratio:",
        (np.dot(solve_qp_problem(gamma).T, mu) - r) / volatilities_function(gamma),
    )

### Question 3.d

In [ ]:
sharpe_ratios = (long_returns - r * 100) / long_volatilities  # Converting r into %

i = np.argmax(sharpe_ratios)
tp = solve_long_only_qp_problem(precise_gammas[i])
print("Optimal portfolio for maximum Sharpe ratio:", tp)
print("Maximum Sharpe ratio:", sharpe_ratios[i])
print("Expected return (in %):", returns[i])
print("Volatility (in %):", volatilities[i])

### Question 3.e

In [ ]:
beta = cov_mat.dot(tp) / tp.T.dot(cov_mat).dot(tp)

for i, value in enumerate(beta):
    print("Beta for asset", i + 1, ":", value)

### Question 4.a-d
Theoretical questions, no code needed.
### Question 4.e

## Equity portfolio optimization with net zero objectives
### Question 1.a

In [ ]:
beta = np.array(
    [0.95, 1.05, 0.45, 1.40, 1.15, 0.75, 1.00, 1.20, 1.10, 0.8, 0.7]
).reshape(-1, 1)
sigma_mat = (
    np.diag(
        [0.262, 0.329, 0.211, 0.338, 0.231, 0.259, 0.265, 0.271, 0.301, 0.274, 0.228]
    )
    ** 2
)

weights = np.array(
    [0.82, 0.123, 0.069, 0.031, 0.132, 0.126, 0.102, 0.23, 0.045, 0.028, 0.032]
)

# Computing covariance matrix
sigma_m = 0.2**2

cov_mat: np.ndarray = sigma_m * (beta.dot(beta.T)) + sigma_mat
cov_mat

In [ ]:
# Computing correlation matrix
corr_mat = np.zeros_like(cov_mat)

for i in range(len(corr_mat)):
    for j in range(i, len(corr_mat)):
        corr_mat[i, j] = cov_mat[i, j] / (np.sqrt(sigma_mat[i, i] * sigma_mat[j, j]))
        corr_mat[j, i] = corr_mat[i, j]

corr_mat

In [ ]:
sector_volatility = np.sqrt(np.diag(cov_mat))
sector_volatility

### Question 1.b

The volatility of the benchmark can be computed as follows:
$$ \sigma_{\text{benchmark}} = \sqrt{b^{\mathsf T} \Sigma b}$$
where $b$ is the vector of the benchmark weights.

In [ ]:
b = np.array(
    [8.20, 12.30, 6.90, 3.10, 13.20, 12.60, 10.20, 23.00, 4.50, 2.80, 3.20]
)  # vector of weights of the benchmark
b = b / 100  # convert from percentage to decimal
sigma_benchmark = np.sqrt((b.T).dot(cov_mat).dot(b))
print("The volatility of the benchmark portfolio is: ", sigma_benchmark)

### Question 1.c

In [ ]:
weighted_average = np.sum(weights * beta)
weighted_average

Some assets have a huge ponderation where others have a smaller one.

### Question 1.d

In [ ]:
implied_risk_premia: np.ndarray = (
    0.25 * cov_mat.dot(weights) / np.sqrt(weights.T.dot(cov_mat).dot(weights))
)
implied_risk_premia

In [ ]:
r_m = 0.25 * 0.2 + 0.03

expected_returns = 0.03 + beta.T * (r_m - 0.03)
expected_returns

### Question 1.e

In [ ]:
sci12 = np.array([24, 54, 47, 434, 19, 21, 105, 23, 559, 89, 1655])

ci = np.sum(weights * sci12)
ci

In [ ]:
cm12 = np.array([-2.8, -7.2, -1.8, -1.5, -8.3, -7.8, -8.5, -4.3, -7.1, -2.7, -9.9])

cm = np.sum(weights * cm12)
cm

In [ ]:
gii = np.array([0, 1.5, 0, 0.7, 0, 0, 2.4, 0.2, 0.8, 1.4, 8.4])

gi = np.sum(weights * gii)
gi

### Question 2.b

In [ ]:
def solve_esg_qp_problem(t: float) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        t (float): t parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    w = cp.Variable(11, "w")  # Portfolio weights

    objective = cp.Minimize(0.5 * cp.quad_form(w, cov_mat))
    constraints = [
        cp.sum(w) == 1,
        0 <= w,
        w <= 1,
        cp.sum(sci12 * w) <= 0.7 * (0.93**t) * ci,
    ]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return w.value

In [ ]:
optimal_pfs12 = {}

for t in (0, 1, 2, 5, 10):

    sol = solve_esg_qp_problem(t)

    print("Optimal portfolio for t =", t, ":", sol)

    tracking_eror_volatility = np.sqrt(
        (sol - weights).T.dot(cov_mat).dot(sol - weights)
    )

    print("Tracking error volatility:", tracking_eror_volatility)

    carbon_intensity = sci12.dot(sol)

    print("Carbon intensity:", carbon_intensity)

    carbon_momentum = cm12.dot(sol)

    print("Carbon momentum:", carbon_momentum)
    green_intensity = gii.dot(sol)

    print("Green intensity:", green_intensity)

    reduction_rate = 1 - (sci12.dot(sol)) / (ci)

    print("Reduction rate:", reduction_rate)
    optimal_pfs12[t] = sol

### Question 2.c

In [ ]:
scii13 = np.array([78, 203, 392, 803, 55, 124, 283, 123, 892, 135, 1867])

ci13 = np.sum(weights * scii13)
ci13

In [ ]:
cmi13 = np.array([-0.8, -1.6, -0.1, -0.2, -1.9, -2.0, -2.5, 2.1, -3.6, -0.8, -6.8])

In [ ]:
def solve_esg_qp_problem13(t: float) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        t (float): t parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    w = cp.Variable(11, "w")  # Portfolio weights

    objective = cp.Minimize(0.5 * cp.quad_form(w, cov_mat))
    constraints = [
        cp.sum(w) == 1,
        0 <= w,
        w <= 1,
        cp.sum(scii13 * w) <= 0.7 * (0.93**t) * ci13,
    ]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return w.value

In [ ]:
optimal_pfs13 = {}

for t in (0, 1, 2, 5, 10):

    sol = solve_esg_qp_problem13(t)

    print("Optimal portfolio for t =", t, ":", sol)

    tracking_eror_volatility = np.sqrt(
        (sol - weights).T.dot(cov_mat).dot(sol - weights)
    )

    print("Tracking error volatility:", tracking_eror_volatility)

    carbon_intensity = scii13.dot(sol)

    print("Carbon intensity:", carbon_intensity)

    carbon_momentum = cmi13.dot(sol)

    print("Carbon momentum:", carbon_momentum)
    green_intensity = gii.dot(sol)

    print("Green intensity:", green_intensity)

    reduction_rate = 1 - (scii13.dot(sol)) / (ci)

    print("Reduction rate:", reduction_rate)
    optimal_pfs13[t] = sol

### Question 2.d

In [ ]:
for key12, key13 in zip(optimal_pfs12, optimal_pfs13):
    implied_expected_return = expected_returns.dot(optimal_pfs12[key12])
    print(
        "Implied expected return for t =",
        key12,
        ":",
        np.sum(implied_expected_return - expected_returns),
    )
    implied_expected_return = expected_returns.dot(optimal_pfs13[key13])
    print(
        "Implied expected return for t =",
        key13,
        ":",
        np.sum(implied_expected_return - expected_returns),
    )

### Question 2.e
We expect the carbon intensity to be the same as at $t=0$ before the rebalancing, as the weights of the assets are the same when no rebalancing occurs.

### Question 3.a

In this question we have a constraint on weights where : 
$$ w_i \geq \frac{b_i}{2}$$

So in this case the QP program corresponds to: 

$$ w^* = \argmin \frac{1}{2} w^\mathsf{T} Qw - w^\mathsf{T}R$$

$$\text{s.t.} \left \{ \begin{array}{ccc} A x & = & B \\ C x & \leq & D \\ x^- & \leq & x & \leq &  x^+ \end{array} \right.$$


**where in our case** : 
- the equality constraint is the budget constraint $ \left( \sum_{i=1}^n w_i = 1  \right) $ : $ A = 1_n^\mathsf{T} $ and $ B = 1 $
- the bounds correspond to the no short-selling restriction (long-only constraint) $ \left( 0 \leq w_i \leq 1 \right) $ : $ w^- = \frac{1}{2}b_n $ and $ w^+ = 1_n $ (with $b_n$ the vector of weights of the benchmark).
- minimmization of the tracking error volatility : $ Q = \Sigma $ and $ R = 0_n $
- the decarbonization constraint : $ C = \mathcal{CI}^\mathsf{T} $ and $ D = \mathcal{CI}^⋆ $

In [ ]:
def solve_esg_qp_3_problem(t: float) -> np.ndarray:
    """Solver for the quadratic programming problem

    Args:
        t (float): t parameter

    Returns:
        np.ndarray: optimal portfolio weights
    """
    w = cp.Variable(11, "w")  # Portfolio weights

    objective = cp.Minimize(0.5 * cp.quad_form(w, cov_mat))
    constraints = [
        cp.sum(w) == 1,
        b <= w,
        w <= 1,
        cp.sum(sci12 * w) <= 0.7 * (0.93**t) * ci,
    ]
    problem = cp.Problem(objective, constraints)

    problem.solve()

    return w.value

Here we just considere the time $t=0$:

In [ ]:
t = 0
sol = solve_esg_qp_3_problem(t)
print("Optimal portfolio for t =", t, ":", sol)
tracking_eror_volatility = np.sqrt((sol - weights).T.dot(cov_mat).dot(sol - weights))
print("Tracking error volatility:", tracking_eror_volatility)
carbon_momentum = cm12.dot(sol)
print("Carbon momentum:", carbon_momentum)
carbon_intensity = sci12.dot(sol)
print("Carbon intensity:", carbon_intensity)
green_intensity = gii.dot(sol)
print("Green intensity:", green_intensity)
reduction_rate = 1 - (ci * sol) / (ci)
print("Reduction rate:", reduction_rate)